In [1]:
from helper_classes import Data
import util as ut

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten,Dropout
from tensorflow.keras.regularizers import l2


import warnings
import logging
from scipy import stats
import numpy as np
import random
import torch
from sklearn.preprocessing import MultiLabelBinarizer
random_state = 1
np.random.seed(random_state)
random.seed(random_state)
warnings.filterwarnings('ignore')

In [2]:
dataset='FB15k-237'
kg_root = 'KGs/'+dataset+'/'
dataset=Data(kg_root)

In [3]:
y=[]
x=[]

entitiy_idx=dict()

sub_obj_pairs= dataset.get_entity_pairs_with_predicates(dataset.train_data)
for s_o_pair, predicates in sub_obj_pairs.items():
    s,o=s_o_pair
    entitiy_idx.setdefault(s, len(entitiy_idx))
    entitiy_idx.setdefault(o, len(entitiy_idx))
    x.append([entitiy_idx[s],entitiy_idx[o]])
    y.append(list(predicates))
x=np.array(x)

In [4]:
print(x[0]) # input
print(y[0]) # output

[0 1]
['/location/country/form_of_government']


In [5]:
x=np.array(x)
binarizer=MultiLabelBinarizer()
y=binarizer.fit_transform(y)
print(x[0]) # input
print(y[0]) # output

[0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [6]:
binarizer.classes_[:10]

array(['/american_football/football_team/current_roster./sports/sports_team_roster/position',
       '/award/award_category/category_of',
       '/award/award_category/disciplines_or_subjects',
       '/award/award_category/nominees./award/award_nomination/nominated_for',
       '/award/award_category/winners./award/award_honor/award_winner',
       '/award/award_category/winners./award/award_honor/ceremony',
       '/award/award_ceremony/awards_presented./award/award_honor/award_winner',
       '/award/award_ceremony/awards_presented./award/award_honor/honored_for',
       '/award/award_nominated_work/award_nominations./award/award_nomination/nominated_for',
       '/award/award_nominee/award_nominations./award/award_nomination/award'],
      dtype=object)

In [7]:
embedding_dim=50

model = Sequential()
model.add(Embedding(len(entitiy_idx), embedding_dim, input_length=2,activity_regularizer=l2(0.1)))
model.add(Flatten())
model.add(Dropout(.5))
model.add(Dense(embedding_dim*3, activation='relu',activity_regularizer=l2(0.1)))
model.add(Dropout(.5))
model.add(Dense(y.shape[1], activation='sigmoid'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 50)             725250    
_________________________________________________________________
flatten (Flatten)            (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 150)               15150     
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 237)               35787     
Total params: 776,187
Trainable params: 776,187
Non-trainable params: 0
__________________________________________________

In [8]:
history = model.fit(x, y, batch_size=1000, epochs= 100, use_multiprocessing=True,verbose=1,shuffle=True)

Train on 246236 samples
Epoch 1/100
246236/246236 [==============================] - 5s 22us/sample - loss: 5.3503 - accuracy: 0.0889
Epoch 2/100
246236/246236 [==============================] - 5s 20us/sample - loss: 4.0611 - accuracy: 0.1962
Epoch 3/100
246236/246236 [==============================] - 4s 17us/sample - loss: 2.6630 - accuracy: 0.6610
Epoch 4/100
246236/246236 [==============================] - 4s 16us/sample - loss: 2.0955 - accuracy: 0.7306
Epoch 5/100
246236/246236 [==============================] - 4s 15us/sample - loss: 1.7935 - accuracy: 0.7680
Epoch 6/100
246236/246236 [==============================] - 4s 15us/sample - loss: 1.6004 - accuracy: 0.7921
Epoch 7/100
246236/246236 [==============================] - 5s 20us/sample - loss: 1.4631 - accuracy: 0.8082
Epoch 8/100
246236/246236 [==============================] - 6s 23us/sample - loss: 1.3636 - accuracy: 0.8189
Epoch 9/100
246236/246236 [==============================] - 6s 24us/sample - loss: 1.2824 - acc

In [9]:
def evaluation(model,binarizer,dataset,triples):
    x_=[]
    y_=[]
    
    hits = []
    ranks=[]
    for i in range(10):
        hits.append([])

    rank_per_relation=dict()
        
    for i in triples:
        s,p,o=i
        x_.append((entitiy_idx[s],entitiy_idx[o]))
        y_.append(p)
    
    
    tensor_pred=torch.from_numpy(model.predict(np.array(x_)))
    
    _, ranked_predictions =tensor_pred.topk(k=len(binarizer.classes_))
    
    ranked_predictions=ranked_predictions.numpy()
    
    
    assert len(ranked_predictions)==len(y_)
    
    classes_=binarizer.classes_.tolist()
        
    for i in range(len(y_)):
        true_relation=y_[i]
        ith_class=classes_.index(true_relation)
        
        rank = np.where(ranked_predictions[i]==ith_class)[0]
        

        rank_per_relation.setdefault(true_relation, []).append(rank+1)
        
        ranks.append(rank+1)
        
        for hits_level in range(10):
            if rank <= hits_level:
                hits[hits_level].append(1.0)
    

    hits=np.array(hits)
    ranks=np.array(ranks)
    print('########## Relation Prediction Results ##########')

    print('Mean Hits @5: {0}'.format(sum(hits[4]) / (float(len(y_)))))
    print('Mean Hits @3: {0}'.format(sum(hits[2]) / (float(len(y_)))))
    print('Mean @1: {0}'.format(sum(hits[0]) / (float(len(y_)))))
    print('Mean rank: {0}'.format(np.mean(ranks)))
    print('Mean reciprocal rank: {0}'.format(np.mean(1. / ranks)))
    
    print('########## Relation Prediction Analysis ##########')
    
    for pred, ranks in rank_per_relation.items():
        ranks=np.array(ranks)
        
        average_hit_at_1=np.sum(ranks==1)/len(ranks)
        average_hit_at_3=np.sum(ranks<=3)/len(ranks)
        average_hit_at_5=np.sum(ranks<=5)/len(ranks)
        
        print('{0}:\t Hits@1:\t{1:.3f}'.format(pred,average_hit_at_1))
        print('{0}:\t Hits@3:\t{1:.3f}'.format(pred,average_hit_at_3))
        print('{0}:\t Hits@5:\t{1:.3f}'.format(pred,average_hit_at_5))
        print('{0}:\t MRR:\t{1:.3f}\t number of occurrence {2}'.format(pred,np.mean(1. / ranks),len(ranks)))
        print('################################')
    

In [10]:
evaluation(model,binarizer,dataset,dataset.train_data)

########## Relation Prediction Results ##########
Mean Hits @5: 0.9977950498869963
Mean Hits @3: 0.9909045807838598
Mean @1: 0.8639803024456572
Mean rank: 1.1859507928633115
Mean reciprocal rank: 0.9264693018164917
########## Relation Prediction Analysis ##########
/location/country/form_of_government:	 Hits@1:	1.000
/location/country/form_of_government:	 Hits@3:	1.000
/location/country/form_of_government:	 Hits@5:	1.000
/location/country/form_of_government:	 MRR:	1.000	 number of occurrence 298
################################
/tv/tv_program/regular_cast./tv/regular_tv_appearance/actor:	 Hits@1:	0.941
/tv/tv_program/regular_cast./tv/regular_tv_appearance/actor:	 Hits@3:	1.000
/tv/tv_program/regular_cast./tv/regular_tv_appearance/actor:	 Hits@5:	1.000
/tv/tv_program/regular_cast./tv/regular_tv_appearance/actor:	 MRR:	0.970	 number of occurrence 1272
################################
/media_common/netflix_genre/titles:	 Hits@1:	0.984
/media_common/netflix_genre/titles:	 Hits@3:	1.000
/me

/baseball/baseball_team/team_stats./baseball/baseball_team_stats/season:	 Hits@1:	1.000
/baseball/baseball_team/team_stats./baseball/baseball_team_stats/season:	 Hits@3:	1.000
/baseball/baseball_team/team_stats./baseball/baseball_team_stats/season:	 Hits@5:	1.000
/baseball/baseball_team/team_stats./baseball/baseball_team_stats/season:	 MRR:	1.000	 number of occurrence 233
################################
/user/alexander/philosophy/philosopher/interests:	 Hits@1:	1.000
/user/alexander/philosophy/philosopher/interests:	 Hits@3:	1.000
/user/alexander/philosophy/philosopher/interests:	 Hits@5:	1.000
/user/alexander/philosophy/philosopher/interests:	 MRR:	1.000	 number of occurrence 100
################################
/people/person/languages:	 Hits@1:	1.000
/people/person/languages:	 Hits@3:	1.000
/people/person/languages:	 Hits@5:	1.000
/people/person/languages:	 MRR:	1.000	 number of occurrence 783
################################
/education/educational_institution/students_graduates./e

In [11]:
evaluation(model,binarizer,dataset,dataset.valid_data)

########## Relation Prediction Results ##########
Mean Hits @5: 0.9981741412758188
Mean Hits @3: 0.9937806687207578
Mean @1: 0.9518429761497205
Mean rank: 1.0796530868424055
Mean reciprocal rank: 0.9726371446156674
########## Relation Prediction Analysis ##########
/film/film/genre:	 Hits@1:	1.000
/film/film/genre:	 Hits@3:	1.000
/film/film/genre:	 Hits@5:	1.000
/film/film/genre:	 MRR:	1.000	 number of occurrence 659
################################
/location/location/time_zones:	 Hits@1:	1.000
/location/location/time_zones:	 Hits@3:	1.000
/location/location/time_zones:	 Hits@5:	1.000
/location/location/time_zones:	 MRR:	1.000	 number of occurrence 129
################################
/people/person/spouse_s./people/marriage/type_of_union:	 Hits@1:	1.000
/people/person/spouse_s./people/marriage/type_of_union:	 Hits@3:	1.000
/people/person/spouse_s./people/marriage/type_of_union:	 Hits@5:	1.000
/people/person/spouse_s./people/marriage/type_of_union:	 MRR:	1.000	 number of occurrence 302

In [12]:
evaluation(model,binarizer,dataset,dataset.test_data)

########## Relation Prediction Results ##########
Mean Hits @5: 0.9971621489382523
Mean Hits @3: 0.9924160876798122
Mean @1: 0.9481358254232313
Mean rank: 1.1041197768861923
Mean reciprocal rank: 0.9701572785883865
########## Relation Prediction Analysis ##########
/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month:	 Hits@1:	1.000
/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month:	 Hits@3:	1.000
/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month:	 Hits@5:	1.000
/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month:	 MRR:	1.000	 number of occurrence 60
################################
/music/performance_role/regular_performances./music/group_membership/group:	 Hits@1:	1.000
/music/performance_role/regular_performances./music/group_membership/group:	 Hits@3:	1.000
/music/performance_role/regular_performances./music/group_membership/group:	 Hits@5:	1.000
/music